In [2]:
import torch
import torch.nn.functional as F
from torch import nn

import torchvision
from torchvision.ops import boxes as box_ops

In [97]:
    preds = [
        torch.FloatTensor(10, 20, 4).random_(0, 1),
        torch.LongTensor(10, 20).random_(0, 21),
        torch.FloatTensor(20, 4).random_(0, 201) / 200
    ]

    targets = {
       'boxes': torch.FloatTensor(10, 12, 4).random_(0, 200) / 200,
       'labels':  torch.LongTensor(10, 12).random_(1, 21),
    }

    preds_loc_delta, preds_conf, anchors = preds
    gt_boxes = targets['boxes']
    labels = targets['labels']


# self.box_similarity(gt_boxes, anchors)
- Iou_mat by gt_box vs anchors 

In [98]:
def box_similarity(truths, anchors):
    IoUs_mat = box_ops.box_iou(truths, anchors)
    
    return IoUs_mat

In [104]:
l = labels[0][:]
b = gt_boxes[0][:, :]
b[:, 2:] += b[:, :2]
anchors[:, 2:] += anchors[:, :2]
print(l.shape, b.shape, anchors.shape)

mqm = box_similarity(b, anchors)

# (obj_N, anchors_N)
# obj_N.max <- mqm.max(dim=1)
# anchor_N.max <- mqm.max(dim=0)

torch.Size([12]) torch.Size([12, 4]) torch.Size([20, 4])


# self.proposal_matcher

In [101]:
class Matcher(object):
    BELOW_LOW_THRESHOLD = -1
    BETWEEN_THRESHOLDS = -2

    def __init__(self, high_threshold, low_threshold, allow_low_quality_matches=False):
        assert low_threshold <= high_threshold
        self.high_threshold = high_threshold
        self.low_threshold = low_threshold
        self.allow_low_quality_matches = allow_low_quality_matches

    def __call__(self, match_quality_matrix):
        matched_vals, matches = match_quality_matrix.max(dim=0)

        if self.allow_low_quality_matches:
            all_matches = matches.clone()
            
        below_low_threshold = matched_vals < self.low_threshold
        between_thresholds = (matched_vals >= self.low_threshold) & ( matched_vals < self.high_threshold )

        matches[below_low_threshold] = Matcher.BELOW_LOW_THRESHOLD
        matches[between_thresholds] = Matcher.BETWEEN_THRESHOLDS
        
        if self.allow_low_quality_matches:
            self.set_low_quality_matches_(matches, all_matches, match_quality_matrix)

        return matches

    def set_low_quality_matches_(self, matches, all_matches, match_quality_matrix):
        highest_quality_foreach_gt, hoge = match_quality_matrix.max(dim=1)
        gt_pred_pairs_of_highest_quality = torch.nonzero( match_quality_matrix == highest_quality_foreach_gt[:, None] )

        pred_inds_to_update = gt_pred_pairs_of_highest_quality[:, 1]
        matches[pred_inds_to_update] = all_matches[pred_inds_to_update]
        

In [114]:
max_tresh = .7
min_tresh = .3
match = Matcher(max_tresh, min_tresh, allow_low_quality_matches=True)

mqm_ids = match(mqm)
matched_gt_boxes = anchors[mqm_ids.clamp(min=0)]

print(mqm_ids)
print(l)
bg_ind = mqm_ids < 0
matched_label = l[mqm_ids]
matched_label[bg_ind] = 0
print(matched_label)

tensor([ 8, -1, -2, -2, -2, -2,  6, 11, -2,  4,  1, -2, -2, -2, -2, -2,  2, -2,
         9, -2])
tensor([ 8, 18,  2, 15,  3,  2, 12, 12, 13,  2,  7, 17])
tensor([13,  0,  0,  0,  0,  0, 12, 17,  0,  3, 18,  0,  0,  0,  0,  0,  2,  0,
         2,  0])


In [55]:
hoge = torch.FloatTensor([[0.8508, 0.7609, 0.8174, 0.6254, 0.7831, 0.7430, 0.7154, 0.3022, 0.4388,
        0.6559, 0.6186, 0.5114], [0.8508, 0.7609, 0.8174, 0.6254, 0.7831, 0.7430, 0.7154, 0.3022, 0.4388,
        0.6559, 0.6186, 0.5114]])
hoge[:, ] # 転置処理

tensor([[0.8174, 0.6254, 0.7831, 0.7430, 0.7154, 0.3022, 0.4388, 0.6559, 0.6186,
         0.5114],
        [0.8174, 0.6254, 0.7831, 0.7430, 0.7154, 0.3022, 0.4388, 0.6559, 0.6186,
         0.5114]])

# compute loss

In [148]:
def log_sum_exp(x):
    """Utility function for computing log_sum_exp while determining
    This will be used to determine unaveraged confidence loss across
    all examples in a batch.
    Args:
        x (Variable(tensor)): conf_preds from conf layers
    """
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x-x_max), 1, keepdim=True)) + x_max

In [313]:
    def compute_loss(preds_loc, preds_conf, match_gt_box, match_gt_label, negapos_ratio=3, device='cpu'):
        pos_anchors = match_gt_label > 0 # [B, A]
        num_batch = preds_loc.shape[0] # B
        num_anchor = preds_loc.shape[1]

        # loss loc
        loc_pos_idx = pos_anchors.unsqueeze(pos_anchors.dim()).expand_as(preds_loc) # [B, A, 4]
        loss_loc = F.smooth_l1_loss(preds_loc[loc_pos_idx].view(-1, 4), match_gt_box[loc_pos_idx].view(-1, 4)) # [BxA, 4], [BxA, 4]

        # loss conf
        num_pos = pos_anchors.long().sum(dim=1, keepdim=True)
        num_hard_nega = negapos_ratio * num_pos
        
        loss_conf_all = F.cross_entropy(preds_conf.view(-1, 21), match_gt_label.view(-1), reduction='none') # [BxA, C] vs [BxA]
        print(loss_conf_all.shape)
        loss_conf_all = loss_conf_all.view(num_batch, -1)
        
        #Hard Negative Mining
        loss_conf_pos = loss_conf_all[pos_anchors].sum()        
        loss_conf_neg = loss_conf_all.clone()
        loss_conf_neg[pos_anchors] = 0
        loss_conf_neg, _ = loss_conf_neg.sort(1, descending=True)
        
        hardness_rank = torch.LongTensor(range(num_anchor)).unsqueeze(0).expand_as(loss_conf_neg).to(device)
        print(hardness_rank.shape)
        hard_negas = hardness_rank < num_hard_nega
        print(pos_anchors.shape, hard_negas.shape, loss_conf_neg.shape)
        loss_conf_hard_neg = loss_conf_neg[hard_negas].sum()
        
        # total loss
        N = num_pos.sum()
        loss_conf = (loss_conf_hard_neg + loss_conf_pos) / N
        
        return loss_loc, loss_conf


In [314]:
# test_load
conf_p = torch.FloatTensor(5, 15, 21).random_(0, 100) / 100
loc_delta = torch.FloatTensor(5, 15, 4).random_(0, 200) / 200
loc_delta[:, :, 2:] += loc_delta[:, :, :2]

conf_t = torch.LongTensor(5, 15).random_(0, 21)
loc_t = torch.FloatTensor(5, 15, 4).random_(0, 200) / 200
loc_t[:, :, 2:] += loc_t[:, :, :2]

print(loc_t.shape, loc_delta.shape)
print(conf_t.shape, conf_p.shape)
loc, conf = compute_loss(loc_delta, conf_p, loc_t, conf_t)
print(loc, conf)

torch.Size([5, 15, 4]) torch.Size([5, 15, 4])
torch.Size([5, 15]) torch.Size([5, 15, 21])
torch.Size([75])
torch.Size([5, 15])
torch.Size([5, 15]) torch.Size([5, 15]) torch.Size([5, 15])
tensor(0.1275) tensor(3.3278)


In [266]:
pos_anchors = torch.Tensor(10, 4).random_(0, 2) # [B, A]
num_anchor = pos_anchors.shape[0]
hoge = range(num_anchor)
print(hoge)

range(0, 10)


# detect_test

In [19]:
def decode(loc, priors, variances):
    """Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded bounding box predictions
    """

    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes

In [72]:
from torchvision.ops import boxes as box_ops 

voc = {
    'class_num': 21,
    'lr_steps': [80000, 100000, 120000],
    'max_iter': 120000,
    'feature_maps': [40, 20, 10, 5, 3, 1],
    'min_dim': 320,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratio': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [.1, .2],
    'name': 'VOC',
}

In [132]:
class Detect(nn.Module):
    def __init__(self, top_k, num_classes, conf_thresh, iou_thresh):
        super(Detect, self).__init__()
        
        self.top_k = top_k
        self.num_classes = num_classes
        self.conf_thresh = conf_thresh
        self.iou_thresh = iou_thresh
        self.variance = voc['variance']
        self.img_size = (voc['min_dim']/320, voc['min_dim']/320)
        self.min_size = 0
        
    def filter_proposals(self, loc_delta, anchors):
        decoded_boxes = decode(loc_delta, anchors, self.variance)
        decoded_boxes = box_ops.clip_boxes_to_image(decoded_boxes, self.img_size)
        keep = box_ops.remove_small_boxes(decoded_boxes, self.min_size)
        decoded_boxes = decoded_boxes[keep]
        
        return decoded_boxes
    
    def forward(self, conf_preds, loc_delta, anchors):
        num_batch = conf_p.shape[0]
        num_anchors = anchors.shape[0]

        preds = torch.zeros(num_batch, num_classes, top_k, 5)

        confidense = conf_p.view(num_batch, num_anchors, num_classes).transpose(2, 1)
        
        for num in range(num_batch):
            decoded_boxes = self.filter_proposals(loc_delta[num], anchors)
            conf_score = confidense[i].clone()
            
            for cat in range(self.num_classes):
                c_mask = conf_score[cat].gt(self.conf_thresh)
                scores = conf_score[cat][c_mask]
                
                if scores.shape[0] == 0:
                    continue
                
                l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)
                boxes = decoded_boxes[l_mask].view(-1, 4)
                keep = nms(boxes, scores, self.iou_thresh)[:top_k]
                
                preds[num, cat, :keep.shape[0]] = torch.cat([scores[keep].unsqueeze(1), boxes[keep]], dim=1)        
        
        return preds

In [133]:
# test_load
conf_p = torch.FloatTensor(5, 15, 21).random_(0, 100) / 100
loc_delta = torch.FloatTensor(5, 15, 4).random_(0, 100) / 200
loc_delta[:, :, 2:] += loc_delta[:, :, :2]

anchors = torch.FloatTensor(15, 4).random_(0, 100) / 100
anchors[:, 2:] += anchors[:, :2]

detect = Detect(5, 21, 0.4, 0.45)
preds = detect(conf_p, loc_delta, anchors)
print(preds)
#print(preds[0].shape, preds[1].shape)
#print(preds[1][0, 0, :], preds[0][0, 0])

tensor([[[[0.9400, 0.0312, 0.0000, 0.4073, 0.5156],
          [0.7900, 0.0000, 0.0000, 0.8417, 1.0000],
          [0.7600, 0.0000, 0.0000, 0.8672, 0.3400],
          [0.6800, 0.0000, 0.0000, 0.1225, 0.7571],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.8900, 0.0000, 0.0000, 1.0000, 0.5375],
          [0.6800, 0.0312, 0.0000, 0.4073, 0.5156],
          [0.5700, 0.0000, 0.0000, 0.1225, 0.7571],
          [0.5600, 0.0000, 0.0000, 0.5874, 1.0000],
          [0.4100, 0.3421, 0.0000, 1.0000, 0.9653]],

         [[0.9800, 0.1091, 0.2457, 1.0000, 1.0000],
          [0.8500, 0.0000, 0.0000, 0.8672, 0.3400],
          [0.5600, 0.1044, 0.0000, 0.5680, 0.5978],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         ...,

         [[0.9400, 0.0000, 0.0000, 0.8417, 1.0000],
          [0.7100, 0.0312, 0.0000, 0.4073, 0.5156],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 

In [67]:
for i in range(num_batch):
    decoded_boxes = decode(loc_delta[i], anchors, [.1, .2])
    conf_score = confidense[i].clone()
    
    for cat in range(num_classes):
        # confidence
        c_mask = conf_score[cat].gt(conf_thresh)
        scores = conf_score[cat][c_mask]
        
        if scores.shape[0] == 0:
            continue
            
        # location
        l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)
        boxes = decoded_boxes[l_mask].view(-1, 4)

        keep = nms(boxes, scores, iou_thresh)[:top_k]
        
        det_cat[i, cat, :keep.shape[0]] = scores[keep]
        det_loc[i, cat, :keep.shape[0]] = boxes[keep]

In [66]:
num_batch = conf_p.shape[0]
num_anchors = anchors.shape[0]

det_cat = torch.zeros(num_batch, num_classes, top_k, 1)
det_loc = torch.zeros(num_batch, num_classes, top_k, 4)

confidense = conf_p.view(num_batch, num_anchors, num_classes).transpose(2, 1)
print(det_cat.shape, det_loc.shape, confidense.shape)

torch.Size([5, 21, 5]) torch.Size([5, 21, 5, 4]) torch.Size([5, 21, 15])


In [68]:
print(det_cat)
print(det_loc)

tensor([[[0.9900, 0.9300, 0.7900, 0.7000, 0.5500],
         [0.9200, 0.9200, 0.8900, 0.8400, 0.4900],
         [0.9700, 0.7200, 0.6700, 0.5300, 0.5000],
         [0.8800, 0.7100, 0.6800, 0.5500, 0.4900],
         [0.9800, 0.7400, 0.7400, 0.6900, 0.5200],
         [0.9000, 0.8400, 0.8400, 0.7200, 0.6600],
         [0.9600, 0.9400, 0.7300, 0.6500, 0.5900],
         [0.9800, 0.9700, 0.9300, 0.9000, 0.6700],
         [0.9600, 0.9300, 0.8900, 0.5900, 0.5500],
         [0.8700, 0.7100, 0.6900, 0.6700, 0.6100],
         [0.8700, 0.8300, 0.8100, 0.7900, 0.5800],
         [0.9100, 0.8700, 0.8200, 0.7300, 0.5900],
         [0.9700, 0.8400, 0.8200, 0.8200, 0.7500],
         [0.9700, 0.9300, 0.8500, 0.6400, 0.4900],
         [0.9300, 0.9000, 0.6700, 0.5400, 0.5300],
         [0.9900, 0.8800, 0.8600, 0.7400, 0.5800],
         [0.9800, 0.9800, 0.9400, 0.6800, 0.2700],
         [0.9000, 0.8800, 0.5700, 0.5700, 0.5200],
         [0.9900, 0.9700, 0.8800, 0.7400, 0.4700],
         [0.8800, 0.8100, 0.670

# assign_targets_to_anchors

In [140]:
import torch
from torch import nn

from torchvision.ops import boxes as box_ops
from ssd.utils import *

class MultiLoss(nn.Module):
    def __init__(self, num_classes=21, high_threshold=.7, low_threshold=.3, variances=[.1, .2], negpos_ratio=3, device='cpu'):
        super(MultiLoss, self).__init__()
        self.num_classes = num_classes
        self.negapos_ratio = negpos_ratio
        self.device = device
        self.variances = variances

        self.box_similarity = box_ops.box_iou
        self.proposal_matcher = Matcher(high_threshold, low_threshold, True)

        self.cross_entropy = nn.CrossEntropyLoss(reduction='none')
        self.l1 = nn.SmoothL1Loss()

    def assign_targets_to_anchors(self, anchors, targets):
        batch_num = len(targets)
        anchors_num = anchors.shape[0]

        matched_gt_boxes = torch.FloatTensor(batch_num, anchors_num, 4)
        matched_gt_labels = torch.LongTensor(batch_num, anchors_num)

        for num, target in enumerate(targets):
            truths = target['boxes']
            labels = target['labels']
            print(truths.shape, labels.shape)

            match_quality_matrix = self.box_similarity(truths, anchors)
            matched_ind = self.proposal_matcher(match_quality_matrix)

            # match_gt_box is gene
            matched_gt_boxes_per_batch = truths[matched_ind.clamp(min=0)]
            matched_gt_boxes_per_batch = encode(matched_gt_boxes_per_batch, anchors, self.variances)

            # match_gt_labels is gene
            bg_ind = matched_ind < 0
            matched_labels_per_batch = labels[matched_ind.clamp(min=0)]
            matched_labels_per_batch[bg_ind] = 0

            matched_gt_boxes[num] = matched_gt_boxes_per_batch
            matched_gt_labels[num] = matched_labels_per_batch

        return matched_gt_labels, matched_gt_boxes

    def compute_loss(self, preds_loc, preds_conf, match_gt_box, match_gt_label):
        pos_anchors = match_gt_label > 0  # [B, A]
        num_batch = preds_loc.shape[0]  # B
        num_anchor = preds_loc.shape[1]

        # compute loc: [BxA, 4] vs [BxA, 4]
        loc_pos_idx = pos_anchors.unsqueeze(pos_anchors.dim()).expand_as(preds_loc)  # [B, A, 4]
        loss_loc = self.l1(preds_loc[loc_pos_idx].view(-1, 4), match_gt_box[loc_pos_idx].view(-1, 4))

        # compute conf: [BxA] <- [BxA, C] vs [BxA]
        loss_conf_all = self.cross_entropy(preds_conf.view(-1, self.num_classes), match_gt_label.view(-1))
        loss_conf_all = loss_conf_all.view(num_batch, -1)

        # Hard Negative Mining
        num_pos = pos_anchors.long().sum(dim=1, keepdim=True)
        num_hard_nega = self.negapos_ratio * num_pos

        loss_conf_pos = loss_conf_all[pos_anchors].sum()
        loss_conf_neg = loss_conf_all.clone()
        loss_conf_neg[pos_anchors] = 0
        loss_conf_neg, _ = loss_conf_neg.sort(1, descending=True)

        hardness_rank = torch.LongTensor(range(num_anchor)).unsqueeze(0).expand_as(loss_conf_neg).to(self.device)
        hard_nega = hardness_rank < num_hard_nega
        loss_conf_hard_neg = loss_conf_neg[hard_nega].sum()

        # total loss
        N = num_pos.sum()
        loss_conf = (loss_conf_hard_neg + loss_conf_pos) / N

        return loss_loc, loss_conf

    def hard_negative_mining(self, loss):
        return loss

    def forward(self, preds, targets):
        preds_loc_delta, preds_conf, anchors = preds

        # gene target_gt_boxes
        matched_gt_labels, matched_gt_boxes = self.assign_targets_to_anchors(anchors, targets)
        loss_loc, loss_conf = self.compute_loss(preds_loc_delta, preds_conf, matched_gt_boxes, matched_gt_labels)

        return loss_loc, loss_conf

if __name__=='__main__':
    preds = [
        torch.FloatTensor(10, 20, 4).random_(0, 200) / 200,
        torch.FloatTensor(10, 20, 21).random_(0, 100) / 100,
        torch.FloatTensor(20, 4).random_(0, 200) / 200
    ]

    targets = [{
       'boxes': torch.FloatTensor(12, 4).random_(0, 200) / 200,
       'labels':  torch.LongTensor(12).random_(1, 21),
    } for i in range(10)]

    loss = MultiLoss().forward(preds, targets)


torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
torch.Size([12, 4]) torch.Size([12])
